# Import Packages

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from web3 import Web3 as w3
import json
import base64
import psycopg2query

In [ ]:
# API Keys
web3_key = # API provider
etherscan_key = # etherscan API key

# Infura HTTP endpoint Ethereum mainnet
w3_client = w3(w3.HTTPProvider(f'https://mainnet.infura.io/v3/{web3_key}'))

In [ ]:
# Check connection
print(w3.isConnected)

# [ethereum_files/ethereum.sh]
## Use EthereumETL to pull raw data

ethereum_files/ethereumetl.sh - specify block range to pull data

# [OPTIONAL]
## Upload to ethereumetl_transactions SQL table (if using PostgreSQL DB)


In [ ]:
# Check for blocks have already been added to avoid duplicate keys
sql = """
SELECT DISTINCT block_number
FROM ethereumetl_transactions
"""
psycopg2query.query(sql)

In [ ]:
ethereumetl_df = pd.read_csv('ethereumetl_files/transactions.csv')
ethereumetl_df['block_timestamp'] = ethereumetl_df['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x))

# Insert dataframe into DB
psycopg2query.insert_df_sql(ethereumetl_df, 'ethereumetl_transactions')

# [tranasction_receipts.js]
## For more efficient extraction of transaction logs, use Asynchronous call in node.js

In [ ]:
txn_receipt = pd.read_csv('transaction_receipt.csv')

## [OPTIONAL]
### Upload to transaction_receipts SQL table

In [ ]:
psycopg2query.insert_df_sql(txn_receipt, 'transaction_receipts')

### Query Logs for Decoding

[PostgreSQL DB]
- Filter by methodId: ethereumetl_transactions - first 10 characters = methodId
- Map ethereumetl_transactions.hash to transaction_receipts.transaction_hash

OR

[PostgreSQL DB]
- Filter by transaction_receipts.event_hash

OR

- Use transaction_receipt.csv

# Decode Logs

In [ ]:
# Packages for decoding

import traceback
from pprint import pprint
from eth_utils import event_abi_to_log_topic, to_hex
from hexbytes import HexBytes
from web3._utils.events import get_event_data
from web3.auto import w3
from functools import lru_cache

In [ ]:
def _sanitize_log(log):    
    for i, topic in enumerate(log['topics']):
        if not isinstance(topic, HexBytes):
            log['topics'][i] = HexBytes(topic)

    if 'address' not in log:
        log['address'] = None

    if 'blockHash' not in log:
        log['blockHash'] = None

    if 'blockNumber' not in log:
        log['blockNumber'] = None

    if 'logIndex' not in log:
        log['logIndex'] = None

    if 'transactionHash' not in log:
        log['transactionHash'] = None

    if 'transactionIndex' not in log:
        log['transactionIndex'] = None

    return log

In [ ]:
# Bytecode decoding functions (idk)

def decode_tuple(t, target_field):
    output = dict()
    for i in range(len(t)):
        if isinstance(t[i], (bytes, bytearray)):
            output[target_field[i]['name']] = to_hex(t[i])
        elif isinstance(t[i], (tuple)):
            output[target_field[i]['name']] = decode_tuple(t[i], target_field[i]['components'])
        else:
            output[target_field[i]['name']] = t[i]
    return output

def decode_list_tuple(l, target_field):
    output = l
    for i in range(len(l)):
        output[i] = decode_tuple(l[i], target_field)
    return output

def decode_list(l):
    output = l
    for i in range(len(l)):
        if isinstance(l[i], (bytes, bytearray)):
            output[i] = to_hex(l[i])
        else:
            output[i] = l[i]
    return output

def convert_to_hex(arg, target_schema):
    """
    utility function to convert byte codes into human readable and json serializable data structures
    """
    output = dict()
    for k in arg:
        if isinstance(arg[k], (bytes, bytearray)):
            output[k] = to_hex(arg[k])
        elif isinstance(arg[k], (list)) and len(arg[k]) > 0:
            target = [a for a in target_schema if 'name' in a and a['name'] == k][0]
            if target['type'] == 'tuple[]':
                target_field = target['components']
                output[k] = decode_list_tuple(arg[k], target_field)
            else:
                output[k] = decode_list(arg[k])
        elif isinstance(arg[k], (tuple)):
            target_field = [a['components'] for a in target_schema if 'name' in a and a['name'] == k][0]
            output[k] = decode_tuple(arg[k], target_field)
        else:
            output[k] = arg[k]
    return output

In [ ]:
@lru_cache(maxsize=None)
def _get_abi(event_contract_address):
    abi_url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={event_contract_address}&apikey={etherscan_key}"
    abi = json.loads(requests.get(abi_url).text)['result']
    abi = json.loads(abi)
    return abi

In [ ]:
def _get_topic2abi(event_contract_address):

     abi = _get_abi(event_contract_address)

     # Filter for only event ABIs from smart contract ABI
     event_abi = [a for a in abi if a['type'] == 'event']

     # Create key value for 32 byte log topic for given event ABI (to be matched with transaction event to capture required ABI)
     topic2abi = {event_abi_to_log_topic(_): _ for _ in event_abi}
     return topic2abi

In [ ]:
def decode_log(df): # dataframe (txn_receipt) or pull from DB using psycopg2query
    event_data = {
        'transaction_hash' : [],
        'event_name' : [],
        'event_hash' : [],
        'event_contract_address' : [],
        'decoded_data' : [],
        'target_schema' : []
    }

    for i in df['logs']:
        for n in i:
            try:
                # Get event contract address from downloaded logs
                event_contract_address = n['address']
                # Get event ABIs of smart contract address
                topic2abi = _get_topic2abi(event_contract_address)
                    
                # Ensure log contains all necessary keys
                _sanitize_log(n)

                # Get the ABI of the event (first HexByte in topic) - Match topic2abi (smart contract event ABI list) with transaction event
                event_abi_txn_event = topic2abi[n['topics'][0]]

                # Get the event name
                evt_name = event_abi_txn_event['name']                
                    
                # Get the event data
                data = get_event_data(w3.codec, event_abi_txn_event, n)['args']
                target_schema = event_abi_txn_event['inputs']
                decoded_data = convert_to_hex(data, target_schema) # convert_to_hex from logs_decoder -- convert bytecode into human readable json serializable data structures

                # Append values to dictionary
                event_data['transaction_hash'].append(n['transactionHash'])
                event_data['event_name'].append(evt_name)
                event_data['event_hash'].append(w3_client.toHex(n['topics'][0]))
                event_data['event_contract_address'].append(n['address'])
                event_data['decoded_data'].append(decoded_data)
                event_data['target_schema'].append(target_schema)

            except:
                # Append values to dictionary
                event_data['transaction_hash'].append(n['transactionHash'])
                event_data['event_name'].append('NoABI')
                event_data['event_hash'].append(n['topics'][0])
                event_data['event_contract_address'].append(n['address'])
                event_data['decoded_data'].append('NoABI')
                event_data['target_schema'].append('NoABI')


    print(f"Events: {len(event_data['transaction_hash'])}")
    return pd.DataFrame(event_data) # returns decoded event data based on logs (from df_)

In [ ]:
# Convert logs into json format
txn_receipt['logs'] = txn_receipt['logs'].apply(lambda x: json.loads(x))

In [ ]:
decoded_logs = decode_log(txn_receipt)